## libraries used throughout the program

In [ ]:
import re
import os
from subprocess import call
import json
import pandas as pd
import math
import sys

## this method is used to split a paragraph into sentences
- this creates one entry per sentence in the json file
- it is also used then to extract one sentence from the original mp3 file

In [ ]:
def splitParagraphIntoSentences(paragraph):
    # preprocess the paragraph first before splitting into sentences
    paragraph = re.sub(r'U\.S\.', 'US', paragraph)
    paragraph = re.sub(r'Mr\.', 'Mr', paragraph)
    paragraph = re.sub(r'Mrs\.', 'Mrs', paragraph)
    paragraph = re.sub(r'Ms\.', 'Ms', paragraph)
    paragraph = re.sub(r'Operator:', 'O:', paragraph)
    paragraph = re.sub(r'\[ph\]', '', paragraph)
    paragraph = re.sub(r'\((\d+\:*)+\)', '', paragraph)
    
#     if the paragraph starts with a Question or Answer tag (Q&A section), temporary store the name ...
#     QnA = re.findall('\<.*\>:\s', paragraph)

    # replace the question and answer tag with Q: and A: respectively
    paragraph = re.sub(r'\<Q.*\>:\s', 'Q: ', paragraph)
    paragraph = re.sub(r'\<A.*\>:\s', 'A: ', paragraph)
    # split the paragraph after every period what is followed by a white space
    sentenceEnders = re.compile('(?<=[\.\!\?])\s')
    sentenceList = sentenceEnders.split(paragraph)
    
#     if len(QnA):
#         add the name later again to restore previous structure
#         sentenceList[0] = QnA[0] + sentenceList[0]

    return sentenceList

## here the text gets preprocessed the first time and stored in a new file in a subfolder
- the original file has a new line after every 11th word
- with preprocessing these newlines are being removed
- this means that one whole conversation of one person gets stored as one paragraph in the new file
## stop the execution after here to make manual adjustments to the text

In [ ]:
def processTextFileToParagraph(path, text_file_name, process_folder, text_file):
    # open the text file for the current mp3 file
    with open(path + text_file_name + '.txt') as f:
        # create a new file for the preprocessed text
        with open(process_folder + text_file + 'PreprocessToParagraphs.txt', "w") as f1:
            # while reading the first file write in the second
            for line in f:
                line.replace(".", ". ")
                # only remove newline characters if the line is not empty
                if line not in ['\n', '\r\n']:
                    f1.write(' ')
                    f1.write(line.rstrip('\n'))
                else:
                    f1.write(line)

## here the text gets processed a second time and stored again in a new file in the same folder
- this step is splitting the paragraph back to sentences
- the json file will include then one sentence per entry

In [ ]:
def processTextFileToSentences(process_folder, text_file):
    # open the just created file with processed text
    with open(process_folder + text_file + "PreprocessToParagraphs.txt") as f:
        # create another file for the second preprocessing step
        with open(process_folder + text_file + 'PreprocessToSentences.txt', "w") as f1:
            for line in f:
                not_needed = ['MANAGEMENT DISCUSSION SECTION',
                             'Wire: Bloomberg Transcripts',
                             'CallSource: ',
                             'For more event information and transcripts',
                             'Q&A',
                             'This transcript may not be 100 percent',
                             'Copyright (c)',
                             'END OF STORY']
                if not any(x in line for x in not_needed) or ' Q&A' in line:
                    # split paragraph into sentences
                    sentences = splitParagraphIntoSentences(line)
                    for s in sentences:
                        if s is not '':
                            # remove double white spaces from the text
                            f1.write(' '.join(s.split()) + '\n')

## This method creates the json file for the earlier specified original voice mp3
#### with the help of aeneas the mp3 gets analyzed and aligned with the preprocessed sentences from the file